<a href="https://colab.research.google.com/github/fernandonakagawa/Python_TCS_100h/blob/main/Aula16_1508_e_17_1608(seg_e_ter).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Atividade 14

Você foi contratado por um banco internacional que irá abrir agências no Brasil. Logo você irá elaborar o
sistema bancário que eles necessitarão para gerir as contas de seus futuros clientes. Considere as classes
a seguir para realizar as atividades:

In [12]:
from abc import ABC, abstractmethod
class Conta(ABC):
  def __init__(self, agencia, numero, nome, cpf, valor = 0):
    self.__agencia = agencia
    self.__numero = numero
    self.__nome = nome
    self.__cpf = cpf
    self.__valor = valor
  def __str__(self):
    s = f"{Conta.tipo_str(self)} - Ag:{self.agencia}| Nº:{self.numero}| "
    s += f"Nome: {self.nome}| Cpf: {self.cpf} | Valor: {self.valor}"
    return s
  @abstractmethod
  def sacar(self, quantia):
    if quantia > self.valor: return False
    else:
      self.__valor -= quantia
      return True

  def depositar(self, quantia):
    self.__valor += quantia
    return True

  def transferir(self, conta, quantia):
    if self.sacar(quantia):
      conta.depositar(quantia)
      return True
    return False


  @property
  def agencia(self): return self.__agencia
  @property
  def nome(self): return self.__nome
  @property
  def cpf(self): return self.__cpf
  @property
  def valor(self): return self.__valor
  @property
  def numero(self): return self.__numero
  @agencia.setter
  def agencia(self, value): self.__agencia = value
  @valor.setter
  def valor(self, value): self.__valor = value

  @staticmethod
  def tipo_str(conta):
    if type(conta) == ContaCorrente: return "Conta Corrente"
    elif type(conta) == ContaPoupanca: return "Conta Poupança"
    elif type(conta) == ContaCorrenteEspecial: return "Conta Corrente Especial"
    return type(conta)

class ContaCorrente(Conta):
  def __init__(self, agencia, numero, nome, cpf, valor = 0):
    super().__init__(agencia, numero, nome, cpf, valor)
  def sacar(self, quantia):
    return super().sacar(quantia)

class ContaPoupanca(Conta):
  def __init__(self, agencia, numero, nome, cpf, juros, valor = 0):
    super().__init__(agencia, numero, nome, cpf, valor)
    self.__juros = juros
  def sacar(self, quantia):
    return super().sacar(quantia)
  def render(self, meses):
    for i in range(meses):
      self.valor += self.valor * self.juros
  @property
  def juros(self): return self.__juros

class ContaCorrenteEspecial(ContaCorrente):
  def __init__(self, agencia, numero, nome, cpf, valor = 0, limite_cheque_especial = 0):
    super().__init__(agencia, numero, nome, cpf, valor)
    self.__limite = limite_cheque_especial
  def sacar(self, quantia):
    if quantia > self.valor + self.limite: return False
    else:
      self.valor -= quantia
      return True
  @property
  def limite(self): return self.__limite

class Agencia:

  numero = 1
  def __init__(self, numero_agencia):
    self.__numero_agencia = numero_agencia
    self.__contas = []
  def __str__(self):
    s = f"---=== Agência {self.numero_agencia} ===---\n"
    s += f"Quantidade de contas: {len(self.__contas)}\n"
    for conta in self.__contas:
      s += f"{conta}\n"
    return s
  def adicionar_conta(self, conta):
    conta.agencia = self.numero_agencia
    self.__contas.append(conta)
    Agencia.numero += 1
  def criar_conta(self, nome, cpf, valor = 0, tipo = None,
                  juros = None, limite_cheque_especial = None):
    agencia = self.numero_agencia
    if tipo is None or tipo is ContaCorrente:
      c = ContaCorrente(agencia, Agencia.numero, nome, cpf, valor)
    elif tipo is ContaPoupanca:
      c = ContaPoupanca(agencia, Agencia.numero, nome, cpf, juros, valor)
    elif tipo is ContaCorrenteEspecial:
      c = ContaCorrenteEspecial(agencia, Agencia.numero, nome, cpf, valor,
                                limite_cheque_especial)
    else:
      raise Exception("Erro. Tipo não suportado.")
    self.adicionar_conta(c)
    return c

  def fechar_conta_numero(self, agencia, numero):
    for conta in self.__contas:
      if conta.agencia == agencia and conta.numero == numero:
        self.__contas.remove(conta)
        return True
    return False

  def fechar_conta_cpf(self, cpf):
    for conta in self.__contas:
      if conta.cpf == cpf:
        self.__contas.remove(conta)
        return True
    return False

  @property
  def numero_agencia(self): return self.__numero_agencia

In [2]:
c = Conta(1, "Fernando", "12312312310", 100)
c.sacar(101)

TypeError: ignored

In [2]:
cc = ContaCorrente(1, 1, "Fernando", "12312312310", 100)
cc.valor

100

In [3]:
cp = ContaPoupanca(2, 1, "Daniel", "12345678910", 0.01, 10000)
print(cp.valor)
cp.render(3)
print(cp.valor)
print(cp)

10000
10303.01
Conta Poupança - Ag:2| Nº:1| Nome: Daniel| Cpf: 12345678910 | Valor: 10303.01


In [4]:
cce = ContaCorrenteEspecial(agencia = 1, numero = 3, nome = "Eduardo", 
                            cpf = "12312345610",
                            valor = 10000, limite_cheque_especial = 1000)
print(cce.valor)
print(cce.sacar(8000))
print(cce.valor)
cce.depositar(500)
print(cce.valor)
print(cce.sacar(8000))
print(cce.valor)
print(cce.sacar(3000))
print(cce.valor)

10000
True
2000
2500
False
2500
True
-500


In [5]:
a = Agencia(1)
a.adicionar_conta(cc)
a.criar_conta(nome = "Diego", cpf = "145678902310", valor = 0, tipo = ContaCorrenteEspecial,
              limite_cheque_especial = 1000)
c = a.criar_conta(nome = "Karen", cpf = "145678902111", valor = 0, tipo = ContaCorrente,
              limite_cheque_especial = 1000)
print(a)

---=== Agência 1 ===---
Quantidade de contas: 3
Conta Corrente - Ag:1| Nº:1| Nome: Fernando| Cpf: 12312312310 | Valor: 100
Conta Corrente Especial - Ag:1| Nº:2| Nome: Diego| Cpf: 145678902310 | Valor: 0
Conta Corrente - Ag:1| Nº:3| Nome: Karen| Cpf: 145678902111 | Valor: 0



In [6]:
a.criar_conta(nome = "Renan", cpf = "145678555222", valor = 0, tipo = ContaPoupanca)
print(a)

---=== Agência 1 ===---
Quantidade de contas: 4
Conta Corrente - Ag:1| Nº:1| Nome: Fernando| Cpf: 12312312310 | Valor: 100
Conta Corrente Especial - Ag:1| Nº:2| Nome: Diego| Cpf: 145678902310 | Valor: 0
Conta Corrente - Ag:1| Nº:3| Nome: Karen| Cpf: 145678902111 | Valor: 0
Conta Poupança - Ag:1| Nº:4| Nome: Renan| Cpf: 145678555222 | Valor: 0



In [7]:
if a.fechar_conta_numero(1, 3): print(a)
else: print("Não foi possível fechar a conta")

---=== Agência 1 ===---
Quantidade de contas: 3
Conta Corrente - Ag:1| Nº:1| Nome: Fernando| Cpf: 12312312310 | Valor: 100
Conta Corrente Especial - Ag:1| Nº:2| Nome: Diego| Cpf: 145678902310 | Valor: 0
Conta Poupança - Ag:1| Nº:4| Nome: Renan| Cpf: 145678555222 | Valor: 0



In [8]:
if a.fechar_conta_numero(1, 10): print(a)
else: print("Não foi possível fechar a conta")

Não foi possível fechar a conta


In [9]:
if a.fechar_conta_cpf("145678555222"): print(a)
else: print("Não foi possível fechar a conta")

---=== Agência 1 ===---
Quantidade de contas: 2
Conta Corrente - Ag:1| Nº:1| Nome: Fernando| Cpf: 12312312310 | Valor: 100
Conta Corrente Especial - Ag:1| Nº:2| Nome: Diego| Cpf: 145678902310 | Valor: 0



In [10]:
if a.fechar_conta_cpf("145678555223"): print(a)
else: print("Não foi possível fechar a conta")

Não foi possível fechar a conta


In [14]:
cc.depositar(100)
print(cc)
print(cce)
cc.transferir(cce, 100)
print(cc)
print(cce)

<class '__main__.ContaCorrente'> - Ag:1| Nº:1| Nome: Fernando| Cpf: 12312312310 | Valor: 200
<class '__main__.ContaCorrenteEspecial'> - Ag:1| Nº:3| Nome: Eduardo| Cpf: 12312345610 | Valor: -500
cce 140331267111312
<class '__main__.ContaCorrente'> - Ag:1| Nº:1| Nome: Fernando| Cpf: 12312312310 | Valor: 100
<class '__main__.ContaCorrenteEspecial'> - Ag:1| Nº:3| Nome: Eduardo| Cpf: 12312345610 | Valor: -500
